In [173]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [219]:
url = 'https://www.pro-football-reference.com/years/2019/fantasy.htm'
fantasy_df = pd.read_html(url)[0]

# headers come in as tuples with name and sub-name: this combines them
def column_rename(name):
    if 'Unnamed' in name[0]:
        return name[1]
    else:
        return '_'.join(name)
    
fantasy_df.columns = fantasy_df.columns.map(column_rename)

# gets rid of extra symbols in player names


# sets index from default to player name
fantasy_df = fantasy_df.set_index('Player')

# gets rid of repeat headers throughout table (there are around 20)
fantasy_df = fantasy_df[fantasy_df['FantPos'] != 'FantPos']

# dropping float rows
del fantasy_df['Rushing_Y/A']
del fantasy_df['Receiving_Y/R']
del fantasy_df['Fantasy_PPR']
del fantasy_df['Fantasy_DKPt']
del fantasy_df['Fantasy_FDPt']

# fills and type-converts numeric columns
def numberize(col):
    col.fillna(0, inplace=True)
    return col.astype('int')

for col in fantasy_df.drop(labels=['Tm', 'FantPos'], axis='columns'):
    fantasy_df[col] = numberize(fantasy_df[col])

# combine 2pts scored and 2pts passed for
fantasy_df['Scoring_2PT'] = fantasy_df['Scoring_2PM'] + fantasy_df['Scoring_2PP']

# delete unneeded rows

del df['Scoring_TD']
del df['Scoring_2PM']
del df['Scoring_2PP']

In [171]:
print(df.head())
print(df.loc['Christian McCaffrey'])

                     Rk   Tm FantPos  Age  Games_G  Games_GS  Passing_Cmp  \
Player                                                                      
Christian McCaffrey   1  CAR      RB   23       16        16            0   
Lamar Jackson         2  BAL      QB   22       15        15          265   
Derrick Henry         3  TEN      RB   25       15        15            0   
Aaron Jones           4  GNB      RB   25       16        16            0   
Ezekiel Elliott       5  DAL      RB   24       16        16            0   

                     Passing_Att  Passing_Yds  Passing_TD  ...  Receiving_Yds  \
Player                                                     ...                  
Christian McCaffrey            2            0           0  ...           1005   
Lamar Jackson                401         3127          36  ...              0   
Derrick Henry                  0            0           0  ...            206   
Aaron Jones                    0            0          

In [170]:
# calculate 0.5ppr points
df['Passing_Yds'].fillna(0, inplace=True)
df['Passing_Yds'] = df['Passing_Yds'].astype('int')
df['Fantasy_HalfPPR'] = df['Passing_Yds']*0.04 + df['Passing_TD']*4 + df['Passing_Int']*(-1)
df['Fantasy_HalfPPR'] += df['Receiving_Rec']*0.5 + df['Rushing_Yds']*0.1 + df['Receiving_Yds']*0.1
df['Fantasy_HalfPPR'] += df['Rushing_TD']*6 + df['Receiving_TD']*6 + df['Scoring_2PT']*2
df['Fantasy_HalfPPR'] += df['Fumbles_FL']*(-2)

In [186]:
with open('all_players.json', 'r') as f:
    all_players = json.load(f)
print(all_players)

['Jehu Chesson', 'Justin Watson', 'Jerry Jeudy', 'James Morgan', 'Alex Erickson', 'Reggie Davis', 'Olamide Zaccheaus', 'Ricky Seals-Jones', 'Josh Jacobs', 'Wes Hills', 'Robert Tonyan', 'Richard Rodgers', 'Kendall Blanton', 'Ty Montgomery', 'Patrick Laird', 'Dare Ogunbowale', 'Isaiah Zuber', 'Dalvin Cook', 'Julio Jones', 'Alec Ingold', 'Anthony Ratliff-Williams', 'Jake Burt', 'Antoine Wesley', 'Joe Flacco', 'Darrynton Evans', 'Michael Pittman Jr.', 'Kendrick Rogers', 'Jordan Jones', "Nick O'Leary", 'JuJu Smith-Schuster', 'Davion Davis', 'J.T. Barrett', 'Marvin Jones Jr.', 'Jeff Driskel', 'Jeff Thomas', 'Miles Sanders', 'Tyler Johnson', 'Scotty Washington', 'Tommy Stevens', 'Napoleon Maxwell', 'Corey Davis', 'Kevin Rader', 'Nate Sudfeld', 'Jesper Horsted', 'Josh Love', 'David Fales', 'Kendall Hinton', 'Tyler Higbee', 'Marquez Callaway', 'Maxx Williams', 'Chase Harrell', 'Senorise Perry', 'Drew Sample', 'Brandon Dillon', 'Case Keenum', 'Brian Lewerke', 'Chris Manhertz', 'Durham Smythe', '

In [596]:
# could check if player is in yahoo db by referencing all_players, but the names don't always match

url = 'https://www.pro-football-reference.com/years/2019/scrimmage.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

anchors = []

for tag in soup.find_all('a'):
    # parent check makes sure only players in table are counted (ex: peyton manning 2019 in "popular" section)
    if tag.parent.name == 'td' and tag.parent['data-stat'] == 'player':
        anchors.append(tag)

link_suffixes = [a.get('href') for a in anchors]
links = sorted(['https://www.pro-football-reference.com'+link for link in link_suffixes])

len(link_suffixes)

556

In [597]:
data = {}

for link in links:
    # find player name
    player_soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    player_name_element = player_soup.find('h1', itemprop='name')
    player_name = player_name_element.text.strip()
    
    # check if player position is eligible in fantasy
    player_position = player_name_element.find_next_siblings()[1].contents[-1]
    player_position_name = player_position.strip()[2:]
    if player_position_name not in ['QB', 'WR', 'RB', 'FB', 'TE']:
        continue
    
    print(player_name, player_position_name)
    get_years_df = pd.read_html(link)[0]
    # first table title for qbs is 'Year', otherwise it is ('Games', 'Year'): this map removes double header
    get_years_df.columns = get_years_df.columns.map(lambda x: x[-1] if type(x) == tuple else x)
    get_years_df['Year'] = get_years_df['Year'].str.replace('*', '')
    get_years_df['Year'] = get_years_df['Year'].str.replace('+', '')
    get_years_df['Year'] = get_years_df['Year'].str.strip()
    years = []
    for year in get_years_df['Year']:
        try:
            # 1st check: makes sure year is a number (discards career and yrs/team)
            # 2nd check: doesn't include rookies (no data to pull from)
            if year.isnumeric() and int(year) < 2020:
                years.append(year)
        except AttributeError:
            # accounts for nan- class 'float', doesn't have .isnumeric method
            pass
    
    # slice removes .htm extension
    year_links = [(year, link[:-4]+'/gamelog/'+year+'/') for year in years]
    
    for year, year_link in year_links:
        try:
            season_df = pd.read_html(year_link)[0]
        # accounts for no tables being on gamelog page: happens when player has no stats for a season
        # ex: derek carrier 2012
        except ValueError:
            season_df = pd.DataFrame([0])
        season_series = season_df.iloc[-1]
        season_data = []
        # get wanted data from series, with defaults of 0
        season_data.append(season_series.get(('Passing', 'Att'), 0))
        season_data.append(season_series.get(('Passing', 'Cmp'), 0))
        season_data.append(season_series.get(('Passing', 'Yds'), 0))
        season_data.append(season_series.get(('Passing', 'TD'), 0))
        season_data.append(season_series.get(('Passing', 'Int'), 0))
        season_data.append(season_series.get(('Rushing', 'Att'), 0))
        season_data.append(season_series.get(('Rushing', 'Yds'), 0))
        season_data.append(season_series.get(('Rushing', 'TD'), 0))
        season_data.append(season_series.get(('Receiving', 'Tgt'), 0))
        season_data.append(season_series.get(('Receiving', 'Rec'), 0))
        season_data.append(season_series.get(('Receiving', 'Yds'), 0))
        season_data.append(season_series.get(('Receiving', 'TD'), 0))
        season_data.append(season_series.get(('Kick Returns', 'TD'), 0))
        season_data.append(season_series.get(('Punt Returns', 'TD'), 0))
        season_data.append(season_series.get(('Fumbles', 'FL'), 0))
        season_data.append(season_series.get(('Scoring', '2PM'), 0))
        # add link suffix
        season_data.append(link.split('https://www.pro-football-reference.com')[-1])
        # append to master dict, using name+year as key
        data[player_name + ' ' + year] = season_data
        
columns = ['PassAtt', 'PassCmp', 'PassYds', 'PassTDs', 'PassInt', 'RushAtt', 'RushYds', 'RushTDs']
columns += ['RecTgt', 'RecRec', 'RecYds', 'RecTDs', 'KRTDs', 'PRTDs', 'FL', '2PT', 'Link']
players_df = pd.DataFrame.from_dict(data, orient='index', columns=columns)
players_df.to_csv('player-data/raw_player_data.csv')
players_df.head()

Ameer Abdullah RB
Davante Adams WR
Josh Adams RB
Jay Ajayi RB
Jordan Akins TE
Mo Alie-Cox TE
Javorius Allen RB
Keenan Allen WR
Kyle Allen QB
Geronimo Allison WR
C.J. Anderson RB
Robby Anderson WR
Mark Andrews TE
JJ Arcega-Whiteside WR
Alex Armah FB
Ryquell Armstead RB
Dan Arnold WR
Marcell Ateman WR
Antony Auclair TE
Tavon Austin WR
Kalen Ballage RB
Peyton Barber RB
Saquon Barkley RB
Kenjon Barner RB
Nick Bawden RB
Cole Beasley WR
Andrew Beck TE
Chad Beebe WR
Blake Bell TE
Josh Bellamy WR
Le'Veon Bell RB
Travis Benjamin WR
Giovani Bernard RB
Braxton Berrios WR
Christian Blake WR
Khari Blasingame RB
David Blough QB
C.J. Board WR
Brandon Bolden RB
Reggie Bonnafon RB
Devontae Booker RB
Mike Boone RB
Kendrick Bourne WR
Tyler Boyd WR
Miles Boykin WR
Nick Boyle TE
Cameron Brate TE
Ben Braunecker TE
Matt Breida RB
Tony Brooks-James RB
A.J. Brown WR
Daniel Brown TE
Fred Brown WR
Jaron Brown WR
John Brown WR
Malcolm Brown RB
Marquise Brown WR
Pharaoh Brown TE
Ventell Bryant WR
Rex Burkhead RB
D

,PassAtt,PassCmp,PassYds,PassTDs,PassInt,RushAtt,RushYds,RushTDs,RecTgt,RecRec,RecYds,RecTDs,KRTDs,PRTDs,FL,2PT,Link
Ameer Abdullah 2015,0.0,0.0,0.0,0.0,0.0,143.0,597.0,2.0,38.0,25.0,183.0,1.0,0.0,0.0,2.0,0.0,/players/A/AbduAm00.htm
Ameer Abdullah 2016,0.0,0.0,0.0,0.0,0.0,18.0,101.0,0.0,5.0,5.0,57.0,1.0,0.0,0.0,0.0,0.0,/players/A/AbduAm00.htm
Ameer Abdullah 2017,0.0,0.0,0.0,0.0,0.0,165.0,552.0,4.0,35.0,25.0,162.0,1.0,0.0,0.0,1.0,0.0,/players/A/AbduAm00.htm
Ameer Abdullah 2018,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,3.0,28.0,0.0,0.0,0.0,1.0,0.0,/players/A/AbduAm00.htm
Ameer Abdullah 2019,0.0,0.0,0.0,0.0,0.0,23.0,115.0,0.0,21.0,15.0,88.0,1.0,0.0,0.0,1.0,0.0,/players/A/AbduAm00.htm


In [573]:
players_df

,PassAtt,PassCmp,PassYds,PassTDs,PassInt,RushAtt,RushYds,RushTDs,RecTgt,RecRec,RecYds,RecTDs,KRTDs,PRTDs,FL,2PT,Link,Scoring_2PP,Fantasy_Pts
Player/Season,,,,,,,,,,,,,,,,,,,
Ameer Abdullah 2015,0,0,0,0,0,143.0,597.0,2.0,38.0,25.0,183.0,1.0,0.0,0,2.0,0.0,/players/A/AbduAm00.htm,0,104.5
Ameer Abdullah 2016,0,0,0,0,0,18.0,101.0,0.0,5.0,5.0,57.0,1.0,0.0,0,0.0,0.0,/players/A/AbduAm00.htm,0,24.3
Ameer Abdullah 2017,0,0,0,0,0,165.0,552.0,4.0,35.0,25.0,162.0,1.0,0.0,0,1.0,0.0,/players/A/AbduAm00.htm,0,111.9
Ameer Abdullah 2018,0,0,0,0,0,1.0,1.0,0.0,4.0,3.0,28.0,0.0,0.0,0,1.0,0.0,/players/A/AbduAm00.htm,0,2.4
Ameer Abdullah 2019,0,0,0,0,0,23.0,115.0,0.0,21.0,15.0,88.0,1.0,0.0,0,1.0,0.0,/players/A/AbduAm00.htm,0,31.8
Davante Adams 2014,0,0,0,0,0,0.0,0.0,0.0,66.0,38.0,446.0,3.0,0.0,0,0.0,0.0,/players/A/AdamDa01.htm,0,81.6
Davante Adams 2015,0,0,0,0,0,0.0,0.0,0.0,94.0,50.0,483.0,1.0,0.0,0,0.0,1.0,/players/A/AdamDa01.htm,0,81.3
Davante Adams 2016,0,0,0,0,0,0.0,0.0,0.0,121.0,75.0,997.0,12.0,0.0,0,1.0,1.0,/players/A/AdamDa01.htm,0,209.2
Davante Adams 2017,0,0,0,0,0,0.0,0.0,0.0,117.0,74.0,885.0,10.0,0.0,0,0.0,0.0,/players/A/AdamDa01.htm,0,185.5


In [536]:
players_df_list = players_df.index.to_list()
players_df_list = list(set([' '.join(x.split()[:-1]) for x in players_df_list]))
print([x for x in players_df_list if x not in all_players])
print(sorted([x for x in all_players if x not in players_df_list], key=lambda n: n.split()[-1]))

[]
['Rodney Adams', 'Bralon Addison', 'Quincy Adeboyejo', 'Salvon Ahmed', 'Brandon Aiyuk', 'Cam Akers', 'Rodney Anderson', 'Drew Anderson', 'Darius Anderson', 'Devin Asiasi', 'George Aston', 'Andre Baccellia', 'Alex Bachman', 'Manasseh Bailey', 'Jake Bargas', 'J.T. Barrett', 'Cameron Batson', 'Marcus Baugh', 'Omar Bayless', 'C.J. Beathard', 'Ryan Becker', 'Nate Becker', 'Reggie Begelton', 'LeVante Bellamy', 'Eno Benjamin', 'Kurt Benkert', 'Trinity Benson', 'Rashod Berry', 'Saeed Blacknall', 'Moritz Boehringer', 'Victor Bolden', 'Nick Bowers', 'Devonte Boyd', "Ja'Marcus Bradley", 'Darius Bradwell', 'Tyler Bray', 'Jake Breeland', 'Beau Brinkley', 'Tony Brown', 'Noah Brown', 'Equanimeous St. Brown', 'Jake Browning', 'Harrison Bryant', 'Hunter Bryant', 'Ian Bunting', 'Joe Burrow', 'Jake Burt', 'Hakeem Butler', 'Emmanuel Butler', 'Jake Butt', 'Lawrence Cager', 'Raymond Calais', 'Marquez Callaway', 'George Campbell', 'Dylan Cantrell', 'Patrick Carr', 'Quintez Cephus', 'Jehu Chesson', 'Dan Ch

In [571]:
#add 2pt passing

players_df = pd.read_csv('player-data/raw_player_data.csv', index_col=0)

for year in range(2000, 2020):
    print(year)
    url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
    
    # find links of all players on sheet
    twopt_soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    twopt_anchors = []
    for tag in twopt_soup.find_all('a'):
        if tag.parent.name == 'td': 
            twopt_anchors.append(tag)
            
    # add 2pt-passing values to main df for active players
    for anchor in twopt_anchors:
        # check if corresponding player for the anchor is in the main df
        matching_links = players_df[players_df['Link'] == anchor.get('href')]
        if len(matching_links) >= 1:
            for tag in anchor.parent.parent.children: #iterator of table row contents
                if tag['data-stat'] == 'two_pt_pass':
                    # player name always selects first year in league— should pick corresponding year
                    player_name = ' '.join(matching_links.iloc[0].name.split()[:-1])
                    row_name = player_name + ' ' + str(year)
                    print(row_name, anchor.get('href'))
                    twopt_value = tag.text if tag.text != '' else 0
                    players_df.at[row_name, '2PT'] += twopt_value

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
Dwayne Allen 2012 /players/A/AlleDw00.htm
2013
Keenan Allen 2013 /players/A/AlleKe00.htm
Tavon Austin 2013 /players/A/AustTa00.htm
Dwayne Allen 2013 /players/A/AlleDw00.htm
C.J. Anderson 2013 /players/A/AndeC.00.htm
2014
C.J. Anderson 2014 /players/A/AndeC.00.htm
Keenan Allen 2014 /players/A/AlleKe00.htm
Dwayne Allen 2014 /players/A/AlleDw00.htm
Davante Adams 2014 /players/A/AdamDa01.htm
Tavon Austin 2014 /players/A/AustTa00.htm
2015
Tavon Austin 2015 /players/A/AustTa00.htm
C.J. Anderson 2015 /players/A/AndeC.00.htm
Javorius Allen 2015 /players/A/AlleJa01.htm
Keenan Allen 2015 /players/A/AlleKe00.htm
Ameer Abdullah 2015 /players/A/AbduAm00.htm
Davante Adams 2015 /players/A/AdamDa01.htm
Jay Ajayi 2015 /players/A/AjayJa00.htm
Dwayne Allen 2015 /players/A/AlleDw00.htm
2016
Jay Ajayi 2016 /players/A/AjayJa00.htm
Davante Adams 2016 /players/A/AdamDa01.htm
Tavon Austin 2016 /players/A/AustTa00.htm
C.J. Anderson 2016 /players/A

In [572]:
players_df.rename_axis('Player/Season', inplace=True)

players_df.fillna(0, inplace=True)

# calculate fantasy total
players_df['Fantasy_Pts'] = players_df['PassYds'] * 0.04 + players_df['PassTDs'] * 4
players_df['Fantasy_Pts'] += players_df['PassInt'] * -1
players_df['Fantasy_Pts'] += (players_df['RushYds'] + players_df['RecYds']) * 0.1
players_df['Fantasy_Pts'] += (players_df['RushTDs'] + players_df['RecTDs']) * 6
players_df['Fantasy_Pts'] += players_df['RecRec'] * 0.5
players_df['Fantasy_Pts'] += (players_df['KRTDs'] + players_df['PRTDs']) * 6
players_df['Fantasy_Pts'] += players_df['FL'] * -2 + players_df['2PT'] * 2

players_df.to_csv('player-data/adjusted_player_data.csv')

players_df.head()

,PassAtt,PassCmp,PassYds,PassTDs,PassInt,RushAtt,RushYds,RushTDs,RecTgt,RecRec,RecYds,RecTDs,KRTDs,PRTDs,FL,2PT,Link,Scoring_2PP,Fantasy_Pts
Player/Season,,,,,,,,,,,,,,,,,,,
Ameer Abdullah 2015,0,0,0,0,0,143.0,597.0,2.0,38.0,25.0,183.0,1.0,0.0,0,2.0,0.0,/players/A/AbduAm00.htm,0,104.5
Ameer Abdullah 2016,0,0,0,0,0,18.0,101.0,0.0,5.0,5.0,57.0,1.0,0.0,0,0.0,0.0,/players/A/AbduAm00.htm,0,24.3
Ameer Abdullah 2017,0,0,0,0,0,165.0,552.0,4.0,35.0,25.0,162.0,1.0,0.0,0,1.0,0.0,/players/A/AbduAm00.htm,0,111.9
Ameer Abdullah 2018,0,0,0,0,0,1.0,1.0,0.0,4.0,3.0,28.0,0.0,0.0,0,1.0,0.0,/players/A/AbduAm00.htm,0,2.4
Ameer Abdullah 2019,0,0,0,0,0,23.0,115.0,0.0,21.0,15.0,88.0,1.0,0.0,0,1.0,0.0,/players/A/AbduAm00.htm,0,31.8


In [345]:
years = set()
for index, row in players_df.iterrows():
    years.add(index.split()[-1])
years = {int(year) for year in years}
print(years)

{2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019}


In [524]:
players_df.filter(like='D.J.', axis=0)
# 2pt is broken- should be 2

,PassAtt,PassCmp,PassYds,PassTDs,PassInt,RushAtt,RushYds,RushTDs,RecTgt,RecRec,RecYds,RecTDs,KRTDs,PRTDs,FL,2PM,Link
D.J. Foster 2016,0.0,0.0,0.0,0.0,0.0,7.0,24.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,/players/F/FostD.01.htm
D.J. Foster 2017,0.0,0.0,0.0,0.0,0.0,6.0,19.0,0.0,28.0,17.0,133.0,0.0,0.0,0.0,0.0,0.0,/players/F/FostD.01.htm
D.J. Foster 2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/players/F/FostD.01.htm


<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

In [395]:
players_df

,Rk,Date,G#,Week,Age,Tm,Unnamed: 6_level_1,Opp,Result,GS,...,Passing_Int,Passing_Rate,Passing_Sk,Passing_Yds.1,Passing_Y/A,Passing_AY/A,Punt Returns_Ret,Punt Returns_Yds,Punt Returns_Y/R,Punt Returns_TD
Ameer Abdullah 2015,NaN,16 Games,NaN,NaN,NaN,NaN,NaN,NaN,7-9-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ameer Abdullah 2016,NaN,2 Games,NaN,NaN,NaN,NaN,NaN,NaN,1-1-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ameer Abdullah 2017,NaN,14 Games,NaN,NaN,NaN,NaN,NaN,NaN,8-6-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ameer Abdullah 2018,NaN,10 Games,NaN,NaN,NaN,NaN,NaN,NaN,5-5-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ameer Abdullah 2019,NaN,16 Games,NaN,NaN,NaN,NaN,NaN,NaN,10-6-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Davante Adams 2014,NaN,16 Games,NaN,NaN,NaN,NaN,NaN,NaN,12-4-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Davante Adams 2015,NaN,13 Games,NaN,NaN,NaN,NaN,NaN,NaN,7-6-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Davante Adams 2016,NaN,16 Games,NaN,NaN,NaN,NaN,NaN,NaN,10-6-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Davante Adams 2017,NaN,14 Games,NaN,NaN,NaN,NaN,NaN,NaN,7-7-0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Davante Adams 2018,NaN,15 Games,NaN,NaN,NaN,NaN,NaN,NaN,6-8-1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [390]:
test = 'https://www.pro-football-reference.com/players/B/BrowMa04.htm'
print(test.split('https://www.pro-football-reference.com')[-1])

['', '/players/B/BrowMa04.htm']
